# バッチ エンドポイントへのデプロイ

健康クリニックは一日中患者の測定を取り、各患者の詳細を別々のファイルに保存すると想像してください。 その後、一晩で糖尿病予測モデルを使用して、その日のすべての患者データをバッチとして処理し、翌朝待つ予測を生成し、糖尿病のリスクがあると予測される患者をフォローアップできるようにします。 Azure Machine Learning では、バッチ エンドポイントを作成して、これを実現できます。そして、それをこの演習で実施します。

## 開始する前に

このノートブックでコードを実行するには、最新バージョンの **azureml-ai-ml** パッケージが必要です。 次のセルを実行して、パッケージがインストールされていることを確認します。

> **注**:
> **azure-ai-ml** パッケージがインストールされていない場合は、`pip install azure-ai-ml` を実行してインストールします。

In [ ]:
## ワークスペースに接続する

必要な SDK パッケージがインストールされているため、ワークスペースに接続できます。

ワークスペースに接続するには、識別子パラメーター (サブスクリプション ID、リソース グループ名、ワークスペース名) が必要です。 リソース グループ名とワークスペース名は既に入力されています。 コマンドを完了するには、サブスクリプション ID のみが必要です。

必要なパラメーターを見つけるには、Studio の右上にあるサブスクリプションとワークスペース名をクリックします。 右側にペインが開きます。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> サブスクリプション ID をコピーし、**YOUR-SUBSCRIPTION-ID** をコピーした値に置き換えます。 </p>

## モデルを登録する

バッチ デプロイでは、ワークスペースに登録されているモデルのみをデプロイできます。 ローカル `model` フォルダーに保存されている MLflow モデルを登録します。 

In [ ]:
## バッチ エンドポイントを作成する

バッチ エンドポイントは、アプリケーションがバッチ スコアリング ジョブをトリガーするために呼び出すことができる HTTPS エンドポイントです。 バッチ エンドポイント名は、Azure リージョン内で一意である必要があります。 `datetime` 関数を使用して、現在の日付と時刻に基づく一意の名前を生成します。 

In [ ]:
`BatchEndpoint` クラスを使用してエンドポイントを作成するには、名前と必要に応じて説明を指定する必要があります。 エンドポイントを作成したら、モデルをエンドポイントにデプロイします。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 重要:  エンドポイントが作成されるまで待ってから続行してください。 スタジオに緑色の通知が表示されます。 </p>

## 配置を作成する

デプロイは、実際の推論を実行するモデルをホストするのに必要なリソースのセットです。 `BatchDeployment` クラスを使用してエンドポイントのデプロイを作成します。 

MLflow モデルをデプロイしているため、スコアリング スクリプトや環境を定義する必要はありません。 Azure Machine Learning によって、これらの資産が自動的に作成されます。 `model` フォルダー内の `MLmodel` ファイルは、モデルの予想される入力と出力を理解するために使用されます。

モデルのデプロイには、次のパラメーターを使用します。

- `name`: デプロイの名前。
- `description`: デプロイが何を表しているのかをさらに明確にする省略可能な説明。
- `endpoint_name`: モデルをデプロイする必要がある、以前に作成したエンドポイントの名前。
- `model`: 登録済みモデルの名前。
- `compute`: デプロイされたモデルを呼び出して予測を生成するときに使用されるコンピューティング。
- `instance_count`: 予測値の生成に使うコンピューティング ノードの数。
- `max_concurrency_per_instance`: コンピューティング ノードあたりのスコアリング スクリプトの最大並列実行数。
- `mini_batch_size`: 1 回のスコアリング スクリプト実行で渡されるファイルの数。
- `output_action`: それぞれの新しい予測は、出力ファイルに新しい行として追加されます。
- `output_file_name`: 予測値が追加されるファイル。
- `retry_settings`: ミニバッチが失敗した場合の設定。
- `logging_level`- ログの詳細レベル。 使用できる値は、`warning`、`info`、および `debug` です。 

次のセルを実行すると、デプロイが構成および作成されます。

In [ ]:
<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 重要:  デプロイが完了するまで待ってから続行してください。 スタジオに緑色の通知が表示されます。 </p>

複数のモデルをエンドポイントにデプロイできます。 既定のデプロイを設定して、バッチ エンドポイントを呼び出すときに既定で使用するモデルを指定できます。

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> 重要:  既定のデプロイが設定されるまで待ってから続行してください。 スタジオに緑色の通知が表示されます。 </p>

## バッチ予測用のデータを準備する

`data` フォルダーには、ラベルのないデータを含む CSV ファイルがあります。 `data` フォルダー内のファイルを指すデータ資産を作成します。これは、バッチ ジョブの入力として使用します。

In [ ]:
## ジョブを送信する

モデルをバッチ エンドポイントにデプロイし、ラベルのないデータ資産を用意したら、エンドポイントを呼び出して、ラベルのないデータに対する予測を生成する準備ができました。

まず、登録されているデータ資産を参照して入力を定義します。 次に、パイプライン ジョブを送信するエンドポイントを呼び出します。 ジョブ URL を使用して、スタジオでこれを監視できます。 このジョブには、予測を取得するための (生成された) スコアリング スクリプトの実行を表す子ジョブが含まれます。

## 結果を取得する

バッチ エンドポイントを呼び出すパイプライン ジョブが完了すると、結果を表示できます。 すべての予測は、既定のデータストアに格納されている `predictions.csv` ファイルに収集されます。 ファイルをダウンロードし、次のセルを実行するとデータを視覚化できます。 

In [ ]:
from azure.ai.ml.entities import BatchEndpoint

# create a batch endpoint
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="A batch endpoint for classifying diabetes in patients",
)

ml_client.batch_endpoints.begin_create_or_update(endpoint)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the endpoint is created before continuing! A green notification should appear in the studio. </p>

## Create the deployment

A deployment is a set of resources required for hosting the model that does the actual inferencing. We will create a deployment for our endpoint using the `BatchDeployment` class. 

Since you're deploying an MLflow model, you don't need a scoring script or define the environment. Azure Machine Learning will automatically create those assets for you. The `MLmodel` file in the `model` folder is used to understand what the expected inputs and outputs are of the model.

You'll deploy a model with the following parameters:

- `name`: Name of the deployment.
- `description`: Optional description to further clarify what the deployment represents.
- `endpoint_name`: Name of the previously created endpoint the model should be deployed to.
- `model`: Name of the registered model.
- `compute`: Compute to be used when invoking the deployed model to generate predictions.
- `instance_count`: Count of compute nodes to use for generating predictions.
- `max_concurrency_per_instance`: Maximum number of parallel scoring script runs per compute node.
- `mini_batch_size`: Number of files passed per scoring script run.
- `output_action`: Each new prediction will be appended as a new row to the output file.
- `output_file_name`: File to which predictions will be appended.
- `retry_settings`: Settings for a mini-batch fails.
- `logging_level`: The log verbosity level. Allowed values are `warning`, `info`, and `debug`. 

Running the following cell will configure and create the deployment.

In [ ]:
from azure.ai.ml.entities import BatchDeployment, BatchRetrySettings
from azure.ai.ml.constants import BatchDeploymentOutputAction

deployment = BatchDeployment(
    name="classifier-diabetes-mlflow",
    description="A diabetes classifier",
    endpoint_name=endpoint.name,
    model=model,
    compute="aml-cluster",
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=2,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level="info",
)
ml_client.batch_deployments.begin_create_or_update(deployment)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the deployment is completed before continuing! A green notification should appear in the studio. </p>

You can deploy multiple models to an endpoint. You can set the default deployment to specify which model should be used by default when calling a batch endpoint.

In [ ]:
endpoint.defaults = {}

endpoint.defaults["deployment_name"] = deployment.name

ml_client.batch_endpoints.begin_create_or_update(endpoint)

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> IMPORTANT! Wait until the default deployment is set before continuing! A green notification should appear in the studio. </p>

## Prepare the data for batch predictions

In the `data` folder you'll find CSV files with unlabeled data. You'll create a data asset that points to the files in the `data` folder, which you'll use as input for the batch job.

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

data_path = "./data"
dataset_name = "patient-data-unlabeled"

patient_dataset_unlabeled = Data(
    path=data_path,
    type=AssetTypes.URI_FOLDER,
    description="An unlabeled dataset for diabetes classification",
    name=dataset_name,
)
ml_client.data.create_or_update(patient_dataset_unlabeled)

In [ ]:
patient_dataset_unlabeled = ml_client.data.get(
    name="patient-data-unlabeled", label="latest"
)

## Submit the job

Now that you have deployed a model to a batch endpoint, and have an unlabeled data asset, you're ready to invoke the endpoint to generate predictions on the unlabeled data.

First, you'll define the input by referring to the registered data asset. Then, you'll invoke the endpoint, which will submit a pipeline job. You can use the job URL to monitor it in the Studio. The job will contain a child job that represents the running of the (generated) scoring script to get the predictions.

In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

input = Input(type=AssetTypes.URI_FOLDER, path=patient_dataset_unlabeled.id)

In [ ]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name, 
    input=input)

ml_client.jobs.get(job.name)

## Get the results

When the pipeline job that invokes the batch endpoint is completed, you can view the results. All predictions are collected in the `predictions.csv` file that is stored in the default datastore. You can download the file and visualize the data by running the following cells. 

In [ ]:
ml_client.jobs.download(name=job.name, download_path=".", output_name="score")

In [ ]:
with open("predictions.csv", "r") as f:
    data = f.read()

In [ ]:
from ast import literal_eval
import pandas as pd

score = pd.DataFrame(
    literal_eval(data.replace("\n", ",")), columns=["file", "prediction"]
)
score